In [1]:
import numpy as np
import torch
# torch.utils.data是PyTorch中用来处理数据的工具包
from torch.utils import data
from d2l import torch as d2l


## 生成数据集
这里这个函数应该是d2l内部自己写好提供的函数

跟刚才自己定义的那个一样

In [14]:
true_w = torch.tensor([2,-3.4])
true_b = 4.2

# 数据集保存在features和labels中
features,labels =d2l.synthetic_data(true_w,true_b,1000)

## 读取数据集


In [12]:
def load_array(data_arrays,batch_size,is_train=True):
    """"构造一个 pytorch数据迭代器"""
    # TensorDataset内定义了传入参数要是 *tensors 
    # *tensors是非关键字参数，所以传入要加*号

    # data.TensorDataset将多个特征和标签数据组合成一个PyTorch的数据集对象
    # 返回一个对象
    # 方便后续进行数据处理和批处理操作。
    dataset = data.TensorDataset(*data_arrays)
    # DataLoader是一个迭代器，每次返回batch_size个样本
    # 创建一个数据迭代器，用于在训练过程中按批次加载和处理数据。

    # 这里用return一次返回全部的数据
    # 用yeild一次返回一个数据，所以差不多啦
    return data.DataLoader(dataset,batch_size,shuffle=is_train)


In [15]:
batch_size=10
data_iter = load_array((features,labels),batch_size)

- 验证迭代器
  -  `iter()`函数将迭代器 `data_iter` 转换为一个迭代器对象
  -  `next()`函数获取该迭代器的下一个元素

In [16]:
next(iter(data_iter))

[tensor([[-1.7617, -1.1056],
         [ 0.2292, -0.3574],
         [-0.2596, -0.0485],
         [ 0.8269,  0.8693],
         [-0.7845, -0.3831],
         [-1.1202, -0.4330],
         [-2.0147, -1.0008],
         [ 0.5543, -0.0742],
         [ 1.7024,  1.1714],
         [-0.2592,  0.8678]]),
 tensor([[4.4314],
         [5.8924],
         [3.8338],
         [2.8894],
         [3.9179],
         [3.4410],
         [3.5808],
         [5.5492],
         [3.6270],
         [0.7493]])]

## 定义模型

- `Sequential`类将多个层串联在一起。
- 当给定输入数据时，`Sequential`实例将数据传入到第一层，
- 然后将第一层的输出作为第二层的输入，以此类推。

- 这里定义了一个全连接层

- 在PyTorch中，全连接层在`Linear`类中定义。
- 值得注意的是，我们将两个参数传递到`nn.Linear`中。
- 第一个指定输入特征形状，即2
- 第二个指定输出特征形状，输出特征形状为单个标量，因此为1。

`nn`是神经网络的缩写

In [17]:
# nn是神经网络的缩写
from torch import nn
net = nn.Sequential(nn.Linear(2,1))

## 初始化模型参数
- 框架里一般都有预定义方法来初始化参数
- 我们通过`net[0]`选择网络中的第一个图层，
- 然后使用`weight.data`和`bias.data`方法访问参数。
- 我们还可以使用替换方法`normal_`和`fill_`来重写参数值。
  - 我们可以通过`_`结尾的方法将参数替换，从而初始化参数。

In [30]:
# normal_()函数将权重参数每个元素初始化为随机采样于均值为0、标准差为0.01的正态分布
net[0].weight.data.normal_(0,0.01)

# 不加后面的下划线，报错'Tensor' object has no attribute 'normal'
# net[0].weight.data.normal(0,0.01)

# bias.data.fill_(0)将偏差参数清零
net[0].bias.data.fill_(0)

tensor([0.])

## 定义损失函数
- 框架都写好了

In [19]:
loss = nn.MSELoss()

## 定义优化算法
- 我们要指定优化的参数
  - 可通过`net.parameters()`从我们的模型中获得
- 小批量随机梯度下降只需要设置`lr`值

In [20]:
trainer =torch.optim.SGD(net.parameters(),lr=0.03)

## 训练
基本一样

In [31]:
num_epochs = 3
for epoch in range(num_epochs):
    for X,y in data_iter:
        l = loss(net(X),y)
        # 梯度清零
        trainer.zero_grad()
        # 反向传播,这里不用.sum()，因为框架实现了
        l.backward()
        trainer.step()
    l = loss(net(features),labels)
    print(f"epoch {epoch+1}, loss {l:f}")
    # 查看梯度
    print(net[0].weight.grad)
    print(net[0].bias.grad)

epoch 1, loss 0.000241
tensor([[-0.0065, -0.0103]])
tensor([-0.0338])
epoch 2, loss 0.000107
tensor([[-0.0027,  0.0009]])
tensor([-0.0009])
epoch 3, loss 0.000109
tensor([[-0.0030,  0.0032]])
tensor([-0.0057])


## 小结
* 我们可以使用PyTorch的高级API更简洁地实现模型。
* 在PyTorch中，`data`模块提供了数据处理工具，`nn`模块定义了大量的神经网络层和常见损失函数。
* 我们可以通过`_`结尾的方法将参数替换，从而初始化参数。

## 练习
- 如何访问线性回归的梯度？
  - 见训练最后一行代码